In [ ]:
import os
import numpy as np
import pickle

codebook_size = 10000

In [ ]:
with open("MFCC/normalizer_fit.pkl", "rb") as f:
    normalizer_fit = pickle.load(f)
    
with open("MFCC/kmeans_%s.pkl" % str(codebook_size), "rb") as f:
    kmeans = pickle.load(f)

In [ ]:
center_num = kmeans.cluster_centers_.shape[0]
print("number of kmeans centers:", center_num)

In [ ]:
csvPath = os.path.abspath("MFCC/csv")
labellst = os.listdir(csvPath)
for label in labellst:
    try:
        intlabel = int(label)
    except:
        labellst.remove(label)

In [ ]:
derived_feature_train = []
derived_label_train = []
derived_feature_test = []
derived_label_test = []

In [ ]:
for label in labellst:
    TrainPath = '%s/%s/train' % (csvPath, label)
    TestPath = '%s/%s/test' % (csvPath, label)
    i, j = 0, 0

    for file in os.listdir(TrainPath):
        filePath = os.path.join(TrainPath, file)
        csv_features = np.loadtxt(filePath, delimiter = ",")
        csv_features = csv_features[~np.isnan(csv_features).any(axis=1)]
        
        # Normalize the csv_features
        norm_features = normalizer_fit.transform(csv_features)
        
        # Get the kmeans output for each file
        kmeans_filter = kmeans.predict(norm_features)
        
        # Count occurrences and build histogram
        unique, counts = np.unique(kmeans_filter, return_counts = True)
        canvas = [0] * center_num
        for i in range(center_num):
            if i in unique:
                canvas[i] = counts[np.where(unique == i)][0]
            
            
        derived_feature_train.append(canvas)
        derived_label_train.append(int(label))

    
    for file in os.listdir(TestPath):
        filePath = os.path.join(TestPath, file)
        csv_features = np.loadtxt(filePath, delimiter = ",")
        csv_features = csv_features[~np.isnan(csv_features).any(axis=1)]
        
        # Normalize the csv_features
        norm_features = normalizer_fit.transform(csv_features)
        
        # Get the kmeans output for each file
        kmeans_filter = kmeans.predict(norm_features)
        
        # Count occurrences and build histogram
        unique, counts = np.unique(kmeans_filter, return_counts = True)
        canvas = [0] * center_num
        for j in range(center_num):
            if j in unique:
                canvas[j] = counts[np.where(unique == j)][0]
            
            
        derived_feature_test.append(canvas)
        derived_label_test.append(int(label))

derived_feature_train = np.array(derived_feature_train).astype(np.float64)
derived_label_train = np.expand_dims(derived_label_train, axis=1).astype(np.int32)
print("derived_feature_train shape:", derived_feature_train.shape)
print("derived_label_train shape:", derived_label_train.shape)
np.savetxt("MFCC/derived_feature_train.csv", derived_feature_train, fmt = '%1.3f', delimiter = ',')
np.savetxt("MFCC/derived_label_train.csv", derived_label_train, delimiter = ',')

derived_feature_test = np.array(derived_feature_test).astype(np.float64)
derived_label_test = np.expand_dims(derived_label_test, axis=1).astype(np.int32)
print("derived_feature_test shape:", derived_feature_test.shape)
print("derived_label_test shape:", derived_label_test.shape)
np.savetxt("MFCC/derived_feature_test.csv", derived_feature_test, fmt = '%1.3f', delimiter = ',')
np.savetxt("MFCC/derived_label_test.csv", derived_label_test, delimiter = ',')

In [ ]:
derived_feature_train

In [ ]:
derived_label_train

In [ ]:
derived_feature_test

In [ ]:
derived_label_test